<img src="logo_UTN.svg" align="right" width="150" /> 

# Solar Panel I-V Curve Tracer

In order to measure the I-V curve of a solar panel, a dynamic load is needed. The way of doing this with a microcontroller is by creating a VCCS (Voltage Controlled Current Source) controlled by the microcontroller.
The best way of doing this is by using an op-amp based circuit. Ideally a microcontroller with a DAC to generate a variable voltage to control the op-amp is preferred.
But not all microcontrollers have a DAC, so a PWM can be used instead. The PWM can be filtered with a low-pass filter to create a DC voltage to control the op-amp.

The op-amp will try to keep the voltage at its inverting input equal to the voltage at its non-inverting input.
Also, the DC voltage is proportional to the duty cycle of the PWM signal. So by changing the duty cycle of the PWM signal, the voltage at the non-inverting input of the op-amp will change and the resulting current through the load will change.

The complete circuit is shown below. A VCCS consists of an op-amp a MOSFET and a shunt resistor. The $V^-$ will be connected to the shunt resistor and the $V^+$ will be connected to the filtered PWM signal.
Both voltages will want to be equal, and the voltage across the shunt resistor is determined by ohm's law.

$$V^- = I_{load} \cdot R_{shunt}$$

$$V^+ = V_{filtered\_PWM} = V_{microcontroller} \, D \, A \quad \text{With A being an attenuation imposed by the filter}$$

$$V^- = V^+ \quad \Rightarrow \quad I_{load} \, R_{shunt} =  V_{microcontroller} \, D \, A \quad \Rightarrow \quad I_{load} = \cfrac{V_{microcontroller} \, A}{R_{shunt}} \, D \quad \text{With $D$ going from 0 to 1}$$

Note: Done with Falstad Simulator


<div align=center>

![Circuit](falstad.svg)

</div>

<div align=center>

![Circuit2](ltspice.png)

</div>

The idea of this notebook is to analyze the transfer function of the low-pass filter used to filter the PWM signal. Also, how the transfer function affects the response of the VCCS. The attenuation of the filter will affect the maximum current that can be drawn from the solar panel.
Moreover, the PWM main frequency must be high enough in order to be filtered properly. If the frequency is too low, the output voltage will have a lot of ripple and the current will not be constant.

An easy way of implementing the low-pass filter is by using two RC sections and an additional resistor to ground at the output. The reason of using two RC sections is to have a steeper roll-off, and the additional resistor to ground is to control the attenuation of the filter.
It's important to note that the last resistor not only controls the attenuation, but also the cut-off frequency of the filter.

## Range By Attenuation

Since the attenuation gives a maximum current, we can think it as a way of setting a measurement range. For example, if we want to measure a solar panel that can deliver a maximum of 5A, we can set the attenuation such that the maximum current is 5A.
It can be done by using the following alternatives:
- Potentiometer as R3
- Different fixed values for R3 that can be switched with a relay or a digital switch (CD4066)
- Digital potentiometer

Maybe the best option is to use a digital potentiometer, since it can be controlled by the microcontroller and can be changed on-the-fly. But it can be more expensive and complex to implement. The measurement ranges should be limited to a few, since the user will have to select the range before measuring.
For this reason, a digital potentiometer would be used with fixed values wasting the point of using a digital potentiometer. A cheap solution could be to use a CD4066 with different fixed values for R3. With this, the user can select between a 5 measurement ranges.

## Range By Duty Cycle Step Size

Another way of setting the measurement range is by changing the step size of the duty cycle. By using a fixed attenuation, the maximum current is fixed. But by changing the step size of the duty cycle, the user can select a range of measurement. This can be done with a considerable resolution, and since the PWM resolution is usually high (8-12 bits), the current resolution can be good enough for most applications. Take into account that the bigger the range, the bigger the step size, deriving in a better resolution. In other words, if we measure near full scale, the resolution will be better than if we measure near zero.

The calculation is simple:

$$\text{step size} = \cfrac{I_{\text{desired range}} \; \text{duty resolution}}{I_{\text{full scale}} \; \text{number of samples}}$$

To set the full scale current, the attenuation must be set. Then, the user can select the desired range and the step size will be calculated. The number of samples is fixed, and it depends on how many points we want to measure in the I-V curve. The duty resolution depends on the microcontroller and the PWM configuration. For example, if we use an 8-bit resolution, the duty resolution is 256. In this case the PWM has a 13-bit resolution, so the duty resolution is 8192.

In [1]:
# Import common packages
from IPython.display import display, Markdown, HTML
import sympy as sp
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal

import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, FloatText
import textwrap

In [2]:
def Z_to_ABCD(Z):
    """
    Convert Z parameters to ABCD parameters.
    """
    Z11, _Z12, Z21, Z22 = Z[0, 0], Z[0, 1], Z[1, 0], Z[1, 1]
    delta_Z = Z.det()
    A = Z11 / Z21
    B = delta_Z / Z21
    C = 1 / Z21
    D = Z22 / Z21
    return sp.Matrix([[A, B], [C, D]])


Z1, Z2 = sp.symbols("Z_1 Z_2")

# TwoPort L network
T_params = sp.Matrix([[(Z1 + Z2) / Z2, Z1], [1 / Z2, 1]])

# TwoPort L network using Z parameters
Z_params = sp.Matrix(
    [
        [Z1 + Z2, Z2],
        [Z2, Z2],
    ]
)

# Verify that both representations are equivalent
assert T_params.equals(Z_to_ABCD(Z_params))

# To calculate two RC networks in cascade, we can multiply their ABCD matrices
C1, C2, R1, R2 = sp.symbols("C_1 C_2 R_1 R_2", positive=True, real=True)
s = sp.symbols("s")
T1 = T_params.subs(Z1, R1).subs(Z2, 1 / (s * C1))
T2 = T_params.subs(Z1, R2).subs(Z2, 1 / (s * C2))
T_total = T1 * T2
# Then we can use an aditional resistor to ground to have more design freedom
R3 = sp.symbols("R_3", positive=True, real=True)
T_load = sp.Matrix([[1, 0], [1 / R3, 1]])

T_final = T_total * T_load
A = T_final[0, 0]

transfer_function = 1 / A
transfer_function = transfer_function.expand()

# normalize denominator to monic and extract omega0/Q if quadratic
H = sp.together(transfer_function)  # single rational
num_sym, den_sym = H.as_numer_denom()  # symbolic numerator/denominator
# expand & clear trivial factors
den_sym = sp.simplify(sp.expand(den_sym))

# make polynomial in s (try to clear any scalar factor so leading coeff = 1 after normalization)
den_poly = sp.Poly(den_sym, s)
# normalize to monic
lc = den_poly.LC()
num_monic = sp.simplify(sp.expand(num_sym / lc))
den_monic = sp.Poly(sp.expand(den_poly.as_expr() / lc), s).as_expr()
# coefficient list [1, a1, a2] for s^2 + a1*s + a2
a_coeffs = sp.Poly(den_monic, s).all_coeffs()
a1 = sp.simplify(a_coeffs[1])
a2 = sp.simplify(a_coeffs[2])
# identify omega0 and Q from s^2 + (omega0/Q) s + omega0^2
omega0 = sp.simplify(sp.sqrt(a2))
Q = sp.simplify(omega0 / a1)
# the numerator is omega_0^2 for a standard second-order system
# we can also identify a gain
k = sp.simplify(num_monic / omega0**2)

markdown_msg = f"""
## Transfer Function Analysis
$$H(s) = \\frac{{1}}{{A}} = {sp.latex(transfer_function)}$$

Making the denominator monic and identifying $\\omega_0$ and $Q$:

$$H(s) = \\frac{{{sp.latex(num_monic)}}}{{{sp.latex(den_monic)}}} = \\frac{{\\omega_0^2}}{{s^2 + (\\frac{{\\omega_0}}{{Q}}) s + \\omega_0^2}}$$

$$ \\omega_0 = {sp.latex(omega0)} $$
$$ Q = {sp.latex(Q)} $$
$$ k = {sp.latex(k)} $$
"""

display(Markdown(markdown_msg))
# display symbolic results


## Transfer Function Analysis
$$H(s) = \frac{1}{A} = \frac{1}{C_{1} C_{2} R_{1} R_{2} s^{2} + \frac{C_{1} R_{1} R_{2} s}{R_{3}} + C_{1} R_{1} s + C_{2} R_{1} s + C_{2} R_{2} s + \frac{R_{1}}{R_{3}} + \frac{R_{2}}{R_{3}} + 1}$$

Making the denominator monic and identifying $\omega_0$ and $Q$:

$$H(s) = \frac{\frac{1}{C_{1} C_{2} R_{1} R_{2}}}{s^{2} + \frac{s \left(C_{1} R_{1} R_{2} + C_{1} R_{1} R_{3} + C_{2} R_{1} R_{3} + C_{2} R_{2} R_{3}\right)}{C_{1} C_{2} R_{1} R_{2} R_{3}} + \frac{R_{1} + R_{2} + R_{3}}{C_{1} C_{2} R_{1} R_{2} R_{3}}} = \frac{\omega_0^2}{s^2 + (\frac{\omega_0}{Q}) s + \omega_0^2}$$

$$ \omega_0 = \frac{\sqrt{R_{1} + R_{2} + R_{3}}}{\sqrt{C_{1}} \sqrt{C_{2}} \sqrt{R_{1}} \sqrt{R_{2}} \sqrt{R_{3}}} $$
$$ Q = \frac{\sqrt{C_{1}} \sqrt{C_{2}} \sqrt{R_{1}} \sqrt{R_{2}} \sqrt{R_{3}} \sqrt{R_{1} + R_{2} + R_{3}}}{C_{1} R_{1} R_{2} + C_{1} R_{1} R_{3} + C_{2} R_{1} R_{3} + C_{2} R_{2} R_{3}} $$
$$ k = \frac{R_{3}}{R_{1} + R_{2} + R_{3}} $$


In [3]:
# If R1 = R2 and C1 = C2, we can simplify the expressions
R = sp.symbols("R", positive=True, real=True)
C = sp.symbols("C", positive=True, real=True)

transfer_function_simpl = (
    transfer_function.subs(R1, R).subs(R2, R).subs(C1, C).subs(C2, C)
)
transfer_function_simpl = sp.simplify(transfer_function_simpl)

Q_simpl = Q.subs(R1, R).subs(R2, R).subs(C1, C).subs(C2, C)
Q_simpl = sp.simplify(Q_simpl)

k_simpl = k.subs(R1, R).subs(R2, R).subs(C1, C).subs(C2, C)
k_simpl = sp.simplify(k_simpl)

omega0_simpl = omega0.subs(R1, R).subs(R2, R).subs(C1, C).subs(C2, C)
omega0_simpl = sp.simplify(omega0_simpl)

markdown_msg = f"""
## Simplified Transfer Function Analysis (R1 = R2 = R and C1 = C2 = C)
$$H(s) = \\frac{{1}}{{A}} = {sp.latex(transfer_function_simpl)}$$
$$ \\omega_0 = {sp.latex(omega0_simpl)} $$
$$ Q = {sp.latex(Q_simpl)} $$
$$ k = {sp.latex(k_simpl)} $$
"""

display(Markdown(markdown_msg))


## Simplified Transfer Function Analysis (R1 = R2 = R and C1 = C2 = C)
$$H(s) = \frac{1}{A} = \frac{R_{3}}{C R^{2} s + 2 R + R_{3} \left(C^{2} R^{2} s^{2} + 3 C R s + 1\right)}$$
$$ \omega_0 = \frac{\sqrt{2 R + R_{3}}}{C R \sqrt{R_{3}}} $$
$$ Q = \frac{\sqrt{R_{3}} \sqrt{2 R + R_{3}}}{R + 3 R_{3}} $$
$$ k = \frac{R_{3}}{2 R + R_{3}} $$


In [4]:
# Define a function to plot the Bode plot of the transfer function
def plot(tf, f_center, f_pwm, center=True):
    num, den = sp.fraction(sp.simplify(tf))
    num_poly = sp.Poly(num, s)
    den_poly = sp.Poly(den, s)
    num_coeffs = [float(c) for c in num_poly.all_coeffs()]
    den_coeffs = [float(c) for c in den_poly.all_coeffs()]

    decades_left = 4
    decades_right = 4
    points_per_decade = 2000
    f_min = f_center / 10**decades_left
    f_max = f_center * 10**decades_right
    frequencies = np.logspace(
        np.log10(f_min),
        np.log10(f_max),
        int(points_per_decade * (decades_left + decades_right)),
    )
    w = 2 * np.pi * frequencies

    system = signal.TransferFunction(num_coeffs, den_coeffs)
    _, mag, phase = signal.bode(system, w=w)
    mag[np.abs(mag) < 1e-6] = 0
    phase[np.abs(phase) < 1e-6] = 0

    fig, ax1 = plt.subplots(figsize=(16, 9))
    ax1.semilogx(frequencies, mag, color="blue")
    ax1.set_xlabel("Frequency (Hz)")
    ax1.set_ylabel("Magnitude (dB)", color="blue")
    ax1.tick_params(axis="y", labelcolor="blue")
    ax1.grid(which="both", linestyle="--", linewidth=0.7)

    ax2 = ax1.twinx()
    ax2.semilogx(frequencies, phase, color="red")
    ax2.set_ylabel("Phase (degrees)", color="red")
    ax2.tick_params(axis="y", labelcolor="red")

    ax1.axvline(f_pwm, color="black", linestyle="--", label="PWM Frequency")
    ax1.legend()
    fig.suptitle("Bode Plot")
    fig.tight_layout()

    if center:
        from io import BytesIO
        import base64

        buf = BytesIO()
        fig.savefig(buf, format="png", dpi=110, bbox_inches="tight")
        buf.seek(0)
        b64 = base64.b64encode(buf.read()).decode("ascii")
        plt.close(fig)
        display(
            HTML(
                f'<div style="text-align:center;"><img src="data:image/png;base64,{b64}" style="max-width:100%; height:auto;"/></div>'
            )
        )
    else:
        plt.show()

In [ ]:
# Then if we set a value to R and C, we can get numerical values
R_val = 10e3
C_val = 100e-9

# Make numeric lambdas
f_omega0 = sp.lambdify((R, C, R3), omega0_simpl, "numpy")
f_Q = sp.lambdify((R, C, R3), Q_simpl, "numpy")
f_tf_sym = sp.lambdify((R, C, R3), transfer_function_simpl, "sympy")
f_k = sp.lambdify((R, C, R3), k_simpl, "numpy")

# The filter is used to filter a PWM signal in order to get a DC voltage proportional to the duty cycle.
# By using the circuit defined at the beginning, we can obtain a proportional current output determined by the Duty cycle and the gain/attenuation (k).
Vdc = sp.symbols("V_{dc}")

# Shunt resistor values
R_shunt = 100e-3
R_current_sensing = 10e-3

Imax = Vdc / R_shunt


# Define the update function for interact()
def update(R3_val):
    # symbolic transfer with current R,C,R3
    tf_sym = sp.simplify(transfer_function_simpl.subs({R: R_val, C: C_val, R3: R3_val}))
    omega0_n = float(f_omega0(R_val, C_val, R3_val))
    Q_n = float(f_Q(R_val, C_val, R3_val))
    k_num = float(f_k(R_val, C_val, R3_val))
    f_0 = omega0_n / (2 * np.pi)

    pwm_f = slider_pwm.value

    markdown_msg = textwrap.dedent(
        f"""
    $$R_3 = {R3_val:.3f}\\ \\Omega $$
    ## Numerical Transfer Function Analysis (R1 = R2 = {R_val/1e3:.2f} kΩ and C1 = C2 = {C_val/1e-9:.2f} nF)

    $$ H(s) = \\frac{{1}}{{A}} = {sp.latex(tf_sym)} $$

    $$ R_3 = {R3_val:.3f}\\ \\Omega $$
    $$ \\omega_0 = {omega0_n:.6f}\\ \\text{{rad/s}} $$
    $$ f_0 = {f_0:.6f}\\ \\text{{Hz}} $$
    $$ Q = {Q_n:.6f} $$
    $$ k = {k_num:.6f} $$
    """
    )
    display(Markdown(markdown_msg))

    tf_numeric = transfer_function_simpl.subs({R: R_val, C: C_val, R3: R3_val})
    plot(tf_numeric, f_0, pwm_f)
    V_in = 3.3
    V_out = k_num * V_in
    Imax_val = Imax.subs(Vdc, V_out)
    markdown_msg = textwrap.dedent(
        f"""
    ## Output Voltage Calculation
    
    $$ V_{{in}} = {V_in} \\text{{ V}} $$
    $$ V_{{out}} = {V_out:.6g} \\text{{ V}} $$
    
    ## Maximum Output Current Calculation
    
    $$ R_{{shunt}} = {R_shunt*1e3:.2f} \\text{{ mΩ}} $$
    $$ I_{{max}} = \\frac{{V_{{dc}}}}{{R_{{shunt}}}} = \\frac{{{V_out:.2f}\\ \\text{{V}}}}{{{R_shunt*1e3:.2f}\\ \\text{{mΩ}}}} = {Imax_val:.2f}\\ \\text{{A}} $$

    ## Output current is controlled by the duty cycle of the PWM signal
    
    $$ I_{{out}} = \\frac{{V_{{out}}}}{{R_{{shunt}}}} = \\frac{{V_{{out}}}}{{R_{{shunt}}}} \; D $$
    
    ## Power dissipated by the shunt resistors at maximum current ($D=1$)
    $$ P_{{shunt}} = I_{{max}}^2 \; R_{{shunt}} = ({Imax_val:.2f}\\ \\text{{A}})^2 \; ({R_shunt*1e3:.2f}\\ \\text{{mΩ}}) = {(Imax_val**2 * R_shunt):.2f}\\ \\text{{W}} $$
    $$ P_{{current\_sensing}} = I_{{max}}^2 \; R_{{current\_sensing}} = ({Imax_val:.2f}\\ \\text{{A}})^2 \; ({R_current_sensing*1e3:.2f}\\ \\text{{mΩ}}) = {(Imax_val**2 * R_current_sensing):.2f}\\ \\text{{W}} $$
    """
    )
    display(Markdown(markdown_msg))


# Simulating a 10k pot
R3_start = 500
slider = FloatSlider(
    min=100, max=10e3, step=100, value=float(R3_start), description="R3 (Ω)"
)
slider_pwm = FloatSlider(
    min=1e3, max=8e3, step=1e3, value=float(8e3), description="PWM (Hz)"
)
text_in = FloatText(value=float(R3_start), description="R3 input (Ω)")

# Hidden control used only by interact (so interact does not show a duplicate input)
hidden = FloatText(value=float(R3_start), layout=widgets.Layout(display="none"))


def on_slider_change(change):
    if change.get("name") == "value":
        hidden.value = change["new"]  # last action = slider


def on_text_change(change):
    if change.get("name") == "value":
        hidden.value = change["new"]  # last action = text box (any float)


def on_text_submit(change):
    if change.get("name") == "value":
        hidden.value = change["new"]  # last action = text box (on submit only)


slider.observe(on_slider_change, names="value")
slider_pwm.observe(on_slider_change, names="value")
text_in.observe(on_text_change, names="value")

display(widgets.VBox([slider, slider_pwm, text_in]))

# Use interact with the hidden widget
_ = interact(update, R3_val=hidden)

interactive(children=(FloatText(value=500.0, description='R3_val', layout=Layout(display='none')), Output()), …

# Conclusion / Final Comments

- The power dissipated by the resistors must be considered.
- The op-amp should be rail-to-rail.
- The MOSFET must be a logic level MOSFET. Like the IRLZ44N.
- The shunt resistor must be a low value resistor with a high power rating. Like a 0.1Ω 5W resistor. The lower the resistance the better, but may be hard to measure the voltage across it.
- The INA219 is a good option for measuring the voltage and current of a solar panel. The Bus Voltage can go up to 26V, the resistor of 10mΩ can measure up to 10A. The default module comes with a 100mΩ resistor and by using the common calibration method, it can measure up to 3.2A. (Maybe another current sensor with higher voltage and current range can be used)
- A digital switch like the CD4066 can be used to switch between different values of R3. This way, the user can select between different measurement ranges. Making it a more versatile instrument, affordable and easy to implement.
- The range can be set by changing the step size of the duty cycle. This way, the user can select a range of measurement without changing the hardware. The resolution will depend on the duty resolution and the number of samples.

# References

- [operational amplifier - MOSFET - OPAMP circuit - Electrical Engineering Stack Exchange](https://electronics.stackexchange.com/questions/57448/mosfet-opamp-circuit)
- [voltage - Driving an IRLZ44N Logic MOSFET with a 2N2222 NPN Transistor from an ESP32 - Electrical Engineering Stack Exchange](https://electronics.stackexchange.com/questions/751783/driving-an-irlz44n-logic-mosfet-with-a-2n2222-npn-transistor-from-an-esp32)
- [microcontroller - Micro-controller controlled current source - Electrical Engineering Stack Exchange](https://electronics.stackexchange.com/questions/56772/micro-controller-controlled-current-source)
- [Unstable Feedback in Opamp+MOSFET circuit for Voltage Controlled Current Source - Electrical Engineering Stack Exchange](https://electronics.stackexchange.com/questions/180175/unstable-feedback-in-opampmosfet-circuit-for-voltage-controlled-current-source)
- [Power MOSFET gate driver fundamentals - AN90059.pdf](https://assets.nexperia.com/documents/application-note/AN90059.pdf)
- [PWM DAC (Rev. A) - slaaec5a.pdf](https://www.ti.com/lit/sd/slaaec5a/slaaec5a.pdf?ts=1756837124081)
- [Using PWM Output as a Digital-to-Analog Converter on a TMS320F280x (Rev. A) - spraa88a.pdf](https://www.ti.com/lit/an/spraa88a/spraa88a.pdf?ts=1756204084617)
- [Using PWM Timer\_B as a DAC (Rev. A) - slaa116a.pdf](https://www.ti.com/lit/an/slaa116a/slaa116a.pdf?ts=1756222437207&ref_url=https%253A%252F%252Fwww.google.com%252F)
- [Dual-Output 8-Bit PWM DAC Using Low-Memory MSP430™ MCUs - slaa804.pdf](https://www.ti.com/lit/ab/slaa804/slaa804.pdf?ts=1756207779110&ref_url=https%253A%252F%252Fwww.ti.com%252Ftool%252FMSP-EXP430FR2311)


